In [5]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

pd.set_option('display.max_rows', 60)
pd.set_option('display.min_rows', 30)
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 40)

%load_ext autoreload
%autoreload 2
# %matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
from tqdm.notebook import tqdm
import zipfile

import urllib.request
from urllib.error import HTTPError

In [7]:
# set start and end year for data
START = 2012
END = 2022

# set root folder to work in 
DATA_DIR = '/media/ed/DATA/Datasets/AirQuality'


def download_data():
    """
    Download an extract the files available at https://aqs.epa.gov/aqsweb/airdata/download_files.html
    """
    
    base_url = 'https://aqs.epa.gov/aqsweb/airdata'
    
    metrics = [44201, 42401, 42101, 42602, 88101, 81102, 'NONOxNOy', 'WIND', 'TEMP', 'PRESS', 'RH_DP']
    
    for metric, year in tqdm([(metric, year) for metric in metrics for year in range(START, END + 1)]):
        
        zip_name = f'daily_{metric}_{year}.zip'
        csv_name = f'daily_{metric}_{year}.csv'
        
        # create 'raw' sub-folder to save raw data to
        os.makedirs(os.path.join(DATA_DIR, 'raw'), exist_ok=True)

        url = os.path.join(base_url, zip_name)
        zip_uri = os.path.join(DATA_DIR, 'raw', zip_name)
        csv_uri = os.path.join(DATA_DIR, 'raw', csv_name)        
        
        # if the csv does not exist, go download the zip file
        if not os.path.isfile(csv_uri):
            
            try:
                
                urllib.request.urlretrieve(url, zip_uri)
                
                # extract zip file to csv
                with zipfile.ZipFile(zip_uri, 'r') as zip_ref:
                    zip_ref.extractall(DATA_DIR)

                os.remove(zip_uri)
                
            except HTTPError as e:
                print(f'{url}: {e}')
        

In [8]:
def clean_data(metric):
    """
    Use the raw data to create a df of clean data
    """
    
    usecols = ['State Code', 'County Code', 'Site Num', 'POC', 'Parameter Name', 'Sample Duration', 'Date Local', 'Arithmetic Mean']
    
    df = pd.concat([pd.read_csv(os.path.join(DATA_DIR, 'raw', f'daily_{metric}_{year}.csv'), usecols=usecols) for year in range(START, END + 1)])
    
    # take California only
    df = df[df['State Code'] == 6]
    
    # for these metrics, select 1H duration
    if metric in [88101, 42401, 42101]:
        df = df[df['Sample Duration'] == '1 HOUR']
    
    # just take RH
    if metric == 'RH_DP':
        df = df[df['Parameter Name'].str.strip() == 'Relative Humidity']

    # just take NO
    if metric == 'NONOxNOy':
        df = df[df['Parameter Name'] == 'Nitric oxide (NO)']
        
    df = df.drop(['State Code', 'Sample Duration', 'Parameter Name'], axis=1)
    
    df['Date Local'] = pd.to_datetime(df['Date Local'])
    
    clean_df = pd.DataFrame()

    for code, subdf in df.groupby(['County Code', 'Site Num', 'POC']):

        subdf = subdf.drop(['County Code', 'Site Num', 'POC'], axis=1).set_index('Date Local', drop=True).rename({'Arithmetic Mean': code}, axis=1)
        
        # some dates duplicated
        subdf = subdf[~subdf.index.duplicated()]
        
        clean_df = pd.concat([clean_df, subdf], axis=1)
        
    return clean_df
    

In [10]:
def save_clean_data():

    metrics = [44201, 42401, 42101, 42602, 88101, 81102, 'NONOxNOy', 'WIND', 'TEMP', 'PRESS', 'RH_DP']

    for metric in metrics:
            
        clean_df = clean_data(metric)
        
        clean_df.to_csv(os.path.join(DATA_DIR, 'processed', f'{metric}.csv'))

In [9]:


urllib.request.urlretrieve('https://incidents.fire.ca.gov/imapdata/mapdataall.csv', os.path.join(DATA_DIR, 'fires.csv'))

('/media/ed/DATA/Datasets/AirQuality/fires.csv',
 <http.client.HTTPMessage at 0x7f1d62ac5090>)

In [13]:
fire = pd.read_csv(os.path.join(DATA_DIR, 'fires.csv'), usecols=['incident_name', 'incident_county', 'incident_acres_burned', 'incident_longitude', 'incident_latitude', 'incident_dateonly_created', 'incident_dateonly_extinguished'])

fire

,incident_name,incident_county,incident_acres_burned,incident_longitude,incident_latitude,incident_dateonly_extinguished,incident_dateonly_created
0,Bridge Fire,Shasta,37.0,-122.309000,40.774000,2018-01-09,2017-10-31
1,Pala Fire,San Diego,122.0,1.000000,1.000000,2009-05-25,2009-05-24
2,River Fire,Inyo,407.0,-118.016510,36.602575,2013-02-28,2013-02-24
3,Fawnskin Fire,San Bernardino,30.0,-116.941311,34.288877,2013-04-22,2013-04-20
4,Gold Fire,Madera,274.0,-119.635004,37.116295,2013-05-01,2013-04-30
5,Panther Fire,Tehama,6896.0,-121.595555,40.190062,2013-05-09,2013-05-01
6,Silverado Fire,Napa,75.0,-122.350844,38.441792,2013-05-01,2013-04-30
7,Yellow Fire,Sonoma,125.0,-122.655616,38.638828,2013-05-03,2013-05-01
8,Summit Fire,Riverside,2956.0,-116.941311,34.288877,2013-05-04,2013-05-01
9,Tres Pinos Fire,San Benito,354.0,-120.937494,37.160346,2013-05-03,2013-05-03


In [14]:
fire.dropna()

,incident_name,incident_county,incident_acres_burned,incident_longitude,incident_latitude,incident_dateonly_extinguished,incident_dateonly_created
0,Bridge Fire,Shasta,37.0,-122.309000,40.774000,2018-01-09,2017-10-31
1,Pala Fire,San Diego,122.0,1.000000,1.000000,2009-05-25,2009-05-24
2,River Fire,Inyo,407.0,-118.016510,36.602575,2013-02-28,2013-02-24
3,Fawnskin Fire,San Bernardino,30.0,-116.941311,34.288877,2013-04-22,2013-04-20
4,Gold Fire,Madera,274.0,-119.635004,37.116295,2013-05-01,2013-04-30
5,Panther Fire,Tehama,6896.0,-121.595555,40.190062,2013-05-09,2013-05-01
6,Silverado Fire,Napa,75.0,-122.350844,38.441792,2013-05-01,2013-04-30
7,Yellow Fire,Sonoma,125.0,-122.655616,38.638828,2013-05-03,2013-05-01
8,Summit Fire,Riverside,2956.0,-116.941311,34.288877,2013-05-04,2013-05-01
9,Tres Pinos Fire,San Benito,354.0,-120.937494,37.160346,2013-05-03,2013-05-03


In [16]:
fire['incident_county'].value_counts()

Riverside                         180
San Diego                         110
San Luis Obispo                    82
Shasta                             77
Butte                              77
Siskiyou                           77
Kern                               76
San Bernardino                     70
Fresno                             68
Los Angeles                        67
Tehama                             62
Monterey                           54
Lassen                             51
Tulare                             48
Lake                               46
                                 ... 
Kern, Ventura                       1
Sonoma, Solano, Lake, Colusa        1
El Dorado, Sacramento               1
Amador, El Dorado                   1
Lake, Napa                          1
Napa, Solano                        1
Napa, Yolo                          1
Glenn, Mendocino                    1
Colusa, Mendocino, Glenn, Lake      1
Colusa, Lake, Mendocino             1
Los Angeles,